In [3]:
import MySQLdb
import json
import gc
import os
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [4]:
# 從config取得DB資料
FilePath = os.getcwd()
with open(FilePath+'\\config.json') as f:
    config = json.load(f)
data = config["passward"]

In [5]:
#連接DB
db = MySQLdb.connect(host = data['host'] , user = data['user'] , passwd = data['passwd'] , db = data['db'] , charset = 'utf8')
#查詢前先建立游標以遊歷資料
cur = db.cursor()             
check = db.cursor()
# checklist 判斷有無儲存過此店家
checklist = []
check.execute("select address from lottery")
for row in check.fetchall():
    checklist.append(row[0])
# SQL取得資料用cur.fetchall來得到data(型態為tuple)


In [6]:
# 呼叫webdriver 
chrome_path = "C:/Users/3c/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Anaconda3 (64-bit)/chromedriver.exe" #chromedriver.exe執行檔所存在的路徑
web = webdriver.Chrome(chrome_path)
web.set_window_position(300,0) #瀏覽器位置
web.set_window_size(1000,1000) #瀏覽器大小

# 開啟瀏覽器連線至指定網頁url
url = 'http://www.taiwanlottery.com.tw/Lotto/se/salelocation.aspx'
web.get(url)


In [7]:

#取得下拉選單all_data
#選取 縣市 的選項
count = 0
s1 = Select(web.find_element_by_name("DropDownList1"))
for num in range(len(s1.options)-1): #迴圈來取得所有data
    s2 = Select(web.find_element_by_name("DropDownList1"))# 因為js原因 選取後會呼叫其他東西導致舊變數無法使用 故需要再次取得一次all_data
    num += 1
    s2.select_by_index(num) # 會執行選取相對應index的option
    # 選取 鄉鎮市區 的選項
    s3 = Select(web.find_element_by_name("DropDownList2"))
    for n in range(len(s3.options)-1):
        s4 = Select(web.find_element_by_name("DropDownList2"))
        s4.select_by_index(n)
        n += 1
        #下拉選單選取完畢 點選查詢
        web.find_element_by_id('Button1').click() 
        soup = BeautifulSoup(web.page_source, "html.parser") 
        for Dealer in soup.select(".tdA_1"): #選取class 
            address , store = Dealer.select("td")[2].text ,Dealer.select("td")[3].text#選取 tag\
            County , Township = Dealer.select("td")[0].text ,Dealer.select("td")[1].text#選取 tag\
            count += 1
            
            if address in checklist:
                continue
            sql = ('INSERT INTO lottery (County, Township, address, store) VALUES (%s,%s,%s,%s)')
            data = (County,Township,address,store)
            cur.execute(sql,data)
            db.commit()
        for Dealer2 in soup.select(".tdA_2"): #選取class 
            address2 , store2 = Dealer2.select("td")[2].text ,Dealer2.select("td")[3].text#選取 tag\
            County2 , Township2 = Dealer2.select("td")[0].text ,Dealer2.select("td")[1].text#選取 tag\
            count += 1
            
            if address2 in checklist:
                continue
            sql2 = ('INSERT INTO lottery (County, Township, address, store) VALUES (%s,%s,%s,%s)')
            data2 = (County2,Township2,address2,store2)
            cur.execute(sql2,data2)
            db.commit()

In [8]:
# 從DB取address資料 自動連到 GoogleMap
AddressXYCount = 0
GoogleMap = "https://www.google.com.tw/maps/place/"
cur = db.cursor()
cur.execute("select address from lottery")
AddressList = cur.fetchall()
db.commit()
try :
    for Address in AddressList :
        NewAddress = Address[0]
        web.get(GoogleMap+NewAddress)
        time.sleep(5)
        CurrentUrl = web.current_url
        XY = CurrentUrl.split("@")[-1]
        X = XY.split(",")[0]
        Y = XY.split(",")[1]
        AddressXYCount = AddressXYCount + 1

        print(AddressXYCount,NewAddress, X, Y)
        #print(messenge)
except Exception as e :
    print(e)

1 台北市中正區中華路二段107巷3號 25.0327958 121.5030768
2 台北市中正區中華路二段303巷4號 25.0292689 121.5028855
3 台北市中正區中華路二段313巷14號 25.0282626 121.5041588
4 台北市中正區中華路二段425號 25.0254291 121.5066994
5 台北市中正區汀州路一段168號 25.0286132 121.5065147
6 台北市中正區汀州路一段252巷1號 25.0270644 121.5084317
7 台北市中正區汀州路二段149號 25.0246534 121.5187201
8 台北市中正區汀州路二段228號 25.0219968 121.5219638
9 台北市中正區汀州路三段285號 25.0122341 121.5334398
10 台北市中正區和平西路一段57號 25.0269924 121.5174784
11 台北市中正區延平南路105號 25.042045 121.5075043
12 台北市中正區忠孝西路一段50-1號（站前地下街Z2區） 23.9587128 120.9761594
Message: chrome not reachable
  (Session info: chrome=81.0.4044.138)

